In [49]:
import numpy as np
import pandas as pd
from IPython.display import display

In [18]:
# Load the gridset

fpv = pd.read_csv("2Dgrid.csv")

### Data Pre-processing

In [19]:
# Describing the gridset

fpv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Temp    1020 non-null   int64  
 1   Pgauge  1020 non-null   float64
 2   Fpv     1020 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 24.0 KB


In [20]:
# Pre-process
print(fpv["Temp"].max())
print(fpv["Temp"].min())

89
70


In [21]:
X_Temp = fpv.loc[:,"Temp"].drop_duplicates(keep="first")
X = X_Temp.to_numpy()
X

array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89], dtype=int64)

In [22]:
print(fpv["Pgauge"].max())
print(fpv["Pgauge"].min())

500.0
0.0


In [23]:
Y_Gauge = fpv.loc[:,["Pgauge"]].drop_duplicates(keep="first")
Y_array = np.array(Y_Gauge)
Y = Y_array.reshape(-1)
Y

array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
       220., 230., 240., 250., 260., 270., 280., 290., 300., 310., 320.,
       330., 340., 350., 360., 370., 380., 390., 400., 410., 420., 430.,
       440., 450., 460., 470., 480., 490., 500.])

In [24]:
# to check whether Z has the require length of array
fpv["Fpv"].value_counts

<bound method IndexOpsMixin.value_counts of 0       1.0000
1       1.0007
2       1.0015
3       1.0022
4       1.0029
         ...  
1015    1.0307
1016    1.0314
1017    1.0321
1018    1.0328
1019    1.0334
Name: Fpv, Length: 1020, dtype: float64>

In [25]:
print(len(X)*len(Y))
print(len(fpv)==len(X)*len(Y))

1020
True


In [26]:
Z_fpv = fpv.loc[:,"Fpv"]
Z_array = np.array(Z_fpv)
Z = Z_array.reshape(-1)
Z

array([1.    , 1.0007, 1.0015, ..., 1.0321, 1.0328, 1.0334])

In [27]:
len(Z)

1020

### Interpolation

In [28]:
#Construct a 2-D grid and interpolate on it:

from scipy import interpolate

FPV = interpolate.interp2d(X, Y, Z, kind='linear')


In [29]:
#Testing Dataset

X1 = np.array ([76.01])
Y1 = np.array ([34.56])

In [30]:
FPVnew = FPV(X1,Y1)

In [31]:
FPVnew

array([1.0024192])

# Loading Raw Data

In [153]:
# Load Raw Data

raw = pd.read_csv("PMEOEVC.csv")

In [154]:
calculationraw = raw[["Consumption (m3)","Pressure (kPa)","Temperature (°C)"]]

In [155]:
#pull m3, P and T from raw into array

calculationrawarray = calculationraw.to_numpy()
    

In [156]:
calculationrawarray

array([[       nan, 2.0000e-01, 1.7010e+01],
       [2.1600e+02, 1.2014e+02, 1.6900e+01],
       [4.1000e+02, 1.3767e+02, 1.6770e+01],
       ...,
       [3.3500e+02, 1.3781e+02, 1.6550e+01],
       [3.4600e+02, 1.3756e+02, 1.7050e+01],
       [3.7300e+02, 1.3747e+02, 1.6720e+01]])

### Calculation for sm3_Predicted using Calculated z-factor

Vectorisation - convert DF on selected columns to Numpy and iterate

In [130]:
#EVC

sm3_predarr = []

for r in CleanRawArray:
    P_term = (r[1]+101.325)/101.325 ##For EVC, Pressure reading is Gauge Pressure. Require Absolute Pressure in the Formula
    T_term = (273.15+15)/(273.15+r[2])
    FPVnew = FPV(r[1],r[2])
    z_1 = FPVnew**2
    sm3_pred = r[0] * T_term * P_term * (1/z_1)
    sm3_predarr = np.append(sm3_predarr,sm3_pred)
    

In [147]:
##Append sm3_pred array to original DF

addedraw = raw

addedraw["Predicted Consumption (Sm3)"] = sm3_predarr.tolist()

In [151]:
finalraw = addedraw[["Date","Time","Consumption (m3)","Pressure (kPa)","Temperature (°C)","Consumption (Sm3)","Predicted Consumption (Sm3)"]]

In [152]:
finalraw

,Date,Time,Consumption (m3),Pressure (kPa),Temperature (°C),Consumption (Sm3),Predicted Consumption (Sm3)
0,24/05/2022,19,NaN,0.20,17.01,NaN,NaN
1,24/05/2022,20,216.0,120.14,16.90,507.0000,468.002120
2,24/05/2022,21,410.0,137.67,16.77,964.0000,959.100749
3,24/05/2022,22,383.0,137.91,17.20,900.0000,895.458120
4,24/05/2022,23,401.0,137.54,17.09,942.0000,936.461495
...,...,...,...,...,...,...,...
972,04/07/2022,7,339.0,138.61,15.67,803.0000,799.286698
973,04/07/2022,8,333.0,138.62,15.79,789.0000,784.833491
974,04/07/2022,9,335.0,137.81,16.55,789.0000,784.734150
975,04/07/2022,10,346.0,137.56,17.05,812.0000,808.202705
